In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd



In [ ]:
# headers содержит требуемые сайтом заголовки указанные в robots.txt сайта

headers = {
    "User-Agent": "bot"
    }
r = requests.get("https://klimat71.ru/cat/cold/cond.htm",headers=headers)

html = BeautifulSoup(r.content, "html.parser")

links = html.find_all("a")

In [ ]:
# Find_link() - принимает на вход ссылку(link) и сохраняет в links_filtered ссылки содержащие (key1) для поиска строк исключая тег можно добавить (key2): and link["href"].find(key2) == -1

def Find_link(link,key1):
  links_filtered = {}
  for link in links:
    if link.has_attr("href") and link["href"].find(key1) > -1 :
      links_filtered[link["href"]] = 1
  return list(links_filtered.keys())


In [ ]:
# список pages сохраняет ссылки на все страницы содержащие требуемые данные.

link = []
pages = []
pages.extend(Find_link(html.find_all("a"),"cond.htm?pg="))
pages=pages[0:2]
print(pages)

['/cat/cold/cond.htm?pg=2', '/cat/cold/cond.htm?pg=3']


In [ ]:
# поочередно проходим каждую страницу и копируем ссылки на все кондицыонеры представленные на ней в список (link). CD между запросами - 1секунда.

for page in pages:
  r=requests.get("https://klimat71.ru/"+page,headers=headers)
  time.sleep(1)
  html=BeautifulSoup(r.content, "html.parser")
  link.extend(Find_link(html.find_all("a"),"/cond.htm?id="))
  print(link)



['/cat/cold/cond.htm?id=12302', '/cat/cold/cond.htm?id=18457', '/cat/cold/cond.htm?id=18468', '/cat/cold/cond.htm?id=18473', '/cat/cold/cond.htm?id=15640', '/cat/cold/cond.htm?id=17753', '/cat/cold/cond.htm?id=20753', '/cat/cold/cond.htm?id=18148', '/cat/cold/cond.htm?id=18458', '/cat/cold/cond.htm?id=18478', '/cat/cold/cond.htm?id=20595', '/cat/cold/cond.htm?id=18167', '/cat/cold/cond.htm?id=11451', '/cat/cold/cond.htm?id=11456', '/cat/cold/cond.htm?id=20489', '/cat/cold/cond.htm?id=20409', '/cat/cold/cond.htm?id=17754', '/cat/cold/cond.htm?id=12303', '/cat/cold/cond.htm?id=18469', '/cat/cold/cond.htm?id=20754']
['/cat/cold/cond.htm?id=12302', '/cat/cold/cond.htm?id=18457', '/cat/cold/cond.htm?id=18468', '/cat/cold/cond.htm?id=18473', '/cat/cold/cond.htm?id=15640', '/cat/cold/cond.htm?id=17753', '/cat/cold/cond.htm?id=20753', '/cat/cold/cond.htm?id=18148', '/cat/cold/cond.htm?id=18458', '/cat/cold/cond.htm?id=18478', '/cat/cold/cond.htm?id=20595', '/cat/cold/cond.htm?id=18167', '/cat/

In [ ]:
# Все полученные ссылки сохраняем отдельным файлом. каждую ссылку отдельной строкой.
n=0
with open("link.txt", "w") as f:
  f.write("\n".join(link))

#data = open("/content/link.txt", "r")

In [ ]:
head = []
parameters = []
data = link[0:1]  # чтобы каждый раз не загружать все ссылки списка link
print(data)


['/cat/cold/cond.htm?id=12302']


In [ ]:
for i in data:
  print(i)

/cat/cold/cond.htm?id=12302


In [ ]:
#цикл перебирает ссылки из data и отправляет гет запросы по каждой из них
for i in data:
    print(i[:-1])
    url = "https://klimat71.ru" + i  # "[:-1]"убирает перенос строки в конце ссылки
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    time.sleep(1)
    n=0
    print("обработано",(data.index(i)+1),"из",len(data))#ждать придется долго - индикатор прогресса
    n+=1

    print (soup)
   


/cat/cold/cond.htm?id=1230
обработано 1 из 1
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="https://www.w3.org/1999/xhtml">
<head>
<title>Кондиционер Lessar LS/LU-H07KPA2 | Купить кондиционеры в Туле с установкой</title>
<base href="https://klimat71.ru/"/><meta content="Lessar LS/LU-H07KPA2 купить бытовые тула установка монтаж заказать сплит системы доставка каталог недорого дешево ремонт" name="keywords"/><meta content="Купить кондиционер Lessar LS/LU-H07KPA2 в Туле в интернет-магазине Klimat71.ru. Широкий выбор кондиционеров всех видов и брендов по отличным ценам! Быстрая доставка, качественная установка кондиционеров (сплит-систем), гарантийное обслуживание." name="description"/><meta content="Sun, 24 Jul 2022 16:15:22 GMT" http-equiv="Last-Modified"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><link href="/fonts/Boblic/stylesheet.css" rel="stylesheet"/>
<link href="http

In [ ]:
print(soup.find("h2"))
data_df = { }
head =[0]
data = []
df0=pd.DataFrame(data=None)

<h2>
			Кондиционер Lessar LS/LU-H07KPA2		</h2>


In [ ]:
#добавляем название кондиционера:
het = ([h2 for h2 in (soup.find("h2"))])
#добавляем параметры кондиционера:
for table in soup.find_all("table", {"class": "param"}):
  for tr in table.find_all('tr'):
    data.append(([td.get_text() for td in tr.find_all('td')]))
df= pd.DataFrame(data = data, columns= ('param','val'))
df_index = pd.Series(df["val"].values, index = df["param"].values)
df_1= pd.DataFrame(df_index, columns = het)

    

MergeError: ignored

In [ ]:
print(data)



[['мощность охлаждения, кВт:', '2,19'], ['мощность обогрева, кВт:', '2,31'], ['рекомендуемая пл-дь (охл./нагр.), м2:', '20'], ['потребляемая мощность (охл./обогр.), кВт:', '0,68/0,64'], ['инверторная регулировка:', 'нет'], ['уровень шума, дБ:', '30/40'], ['габариты внутр./внешн. блока, мм:', '715x250x188/700x540x240'], ['параметры электропитания, В/Ф/ГЦ:', '220/1/50'], ['макс. длина трассы, м:', '20'], ['размер труб (газ/жидкость), мм/дюйм:', '9,52 (3/8") / 6,35 (1/4")'], ['хладагент:', 'R410A'], ['производитель:', 'Китай'], ['гарантия:', '3 года']]


                                                                   0
мощность охлаждения, кВт:                                       2,19
мощность обогрева, кВт:                                         2,31
рекомендуемая пл-дь (охл./нагр.), м2:                             20
потребляемая мощность (охл./обогр.), кВт:                  0,68/0,64
инверторная регулировка:                                         нет
уровень шума, дБ:                                              30/40
габариты внутр./внешн. блока, мм:            715x250x188/700x540x240
параметры электропитания, В/Ф/ГЦ:                           220/1/50
макс. длина трассы, м:                                            20
размер труб (газ/жидкость), мм/дюйм:       9,52 (3/8") / 6,35 (1/4")
хладагент:                                                     R410A
производитель:                                                 Китай
гарантия:                                                     3 года


In [ ]:
with open('data.csv','w') as f:
    for row in head:
        for x in row:
            f.write(str(x) + ',')
        f.write('\n')